# Course Project: Galaxy Classifier

### Lexy Andershock, Gian Fernandez-Aleman, David Long

#### Introduction
This course project for our Intro to Machine Learning course (COSC325) utilizes a logistic regression model on a dataset of colored galaxy images (Galaxy10 SDSS). While the dataset was originally used with a CNN (a more appropriate model for image data), we wanted to experiment and see what we could do with it using one of the learning techniques we've learned in class.

#### Who is this for?
Our target audience for this project would include astronomers, physicists, space enthusiasts, space researchers, educators, and general populations with an interest in galaxy-shape indentification.

#### Purpose of the Project
The problem we are trying to solve is identifying the shapes of newly discovered galaxies. This dataset contains 10 unique categories of different shapes galaxies possess, and we want to train a model to be able to assign one of these categories to new images of galaxies it hasn't encountered yet. With new space telescopes such as JWST, we are discovering new galaxies at an accelerating rate, and being able to quickly classify them for studying would be very useful instead of manually doing it. This will make users' lives significantly better as they can have a system that handles massive amounts of galaxy photosets and produces accurate results, which is more time-efficient as opposed to manually determining the galaxy shapes.

### 1. Setting Up The Environment
We must ensure that we have imported all the appropriate libraries that we will utilize for our data manipulation and modeling.

In [3]:
# Import required libraries
import h5py # !pip install h5py
import numpy as np
import matplotlib.pyplot as plt
import requests

# Scikit-learn libraries and routines
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle

# Document how to download + import the dataset
url = "https://zenodo.org/records/10844811/files/Galaxy10.h5"
file_name = "Galaxy10.h5"

response = requests.get(url, stream=True)
with open(file_name, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print("Download complete!")

Download complete!


### 2. Organizing Data
We then extract our data from the dataset and organize them into two separate categories: images and labels.

In [2]:
# To get the images and labels from file
with h5py.File(file_name, 'r') as data:
    images = np.array(data['images'])
    labels = np.array(data['ans'])

# Flatten images
n_samples, height, width, channels = images.shape
X = images.reshape(n_samples, height * width * channels)
y = labels

# Normalize pixel values between 0 and 1
X = X.astype(np.float32) / 255.0

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'Galaxy10.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### 3. Splitting Data
Once we have the data put into separate categories, we split it into training and testing categories.

In [41]:
# Shuffle the full dataset before splitting
X, y = shuffle(X, y, random_state = 42)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, stratify = y, random_state = 42)

### 4. Scaling Features
We'll scale the features so that logistic regression converges better

In [42]:
# Create and fit the scaler on training data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Use the same scaler to transform the test set
X_test = scaler.transform(X_test)

### 5. Training The Model!
We'll start training the logistic regression model

In [43]:
# Use a smaller subset (full took atleast 9 minutes)
X_train_small = X_train[:7000]
y_train_small = y_train[:7000]

model = OneVsRestClassifier(SGDClassifier(loss = 'log_loss', max_iter = 500))
model.fit(X_train_small, y_train_small)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


OneVsRestClassifier(estimator=SGDClassifier(loss='log_loss', max_iter=500))

### 6. Predicting Using The Model
We'll predict on the test set using the model we just trained

Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.17      0.23       692
           1       0.39      0.54      0.45      1399
           2       0.40      0.43      0.42      1258
           3       0.13      0.10      0.11        70
           4       0.45      0.48      0.47       307
           5       0.00      0.00      0.00         4
           6       0.47      0.44      0.46       118
           7       0.32      0.15      0.21       224
           8       0.29      0.14      0.19       181
           9       0.02      0.01      0.01       104

    accuracy                           0.39      4357
   macro avg       0.28      0.25      0.25      4357
weighted avg       0.37      0.39      0.37      4357



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [44]:
# Predict on the test set
y_pred = model.predict(X_test)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()